In [51]:
#interactive data visualizationb for geopoints: 
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
# A SparkSession can be used create DataFrame, register DataFrame as tables, execute SQL over tables, cache tables, and read parquet files. 
# It is the entry point to programming Spark with the DataFrame API. 
# We can create a SparkSession, usfollowing builder pattern:
import folium
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd
spark2 = SparkSession.builder.getOrCreate()

from pyspark.sql.types import  (StructType, 
                                StructField, 
                                DateType, 
                                BooleanType,
                                DoubleType,
                                IntegerType,
                                StringType,
                               TimestampType)

# We can let Spark infer the schema of our csv data but proving pre-defined schema makes the reading process faster. 
#Further,it helps us to make the colum names to have the format we want, for example, to avoid spaces in the names of the columns.
crimes_schema = StructType([StructField("ID", StringType(), True),
                            StructField("CaseNumber", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("PrimaryType", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("LocationDescription", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", StringType(), True),
                            StructField("District", StringType(), True),
                            StructField("Ward", StringType(), True),
                            StructField("CommunityArea", StringType(), True),
                            StructField("FBICode", StringType(), True ),
                            StructField("XCoordinate", DoubleType(), True),
                            StructField("YCoordinate", DoubleType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("UpdatedOn", DateType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])


crimes2 = spark2.read.csv("C:/lab/crimes10K.csv",
                       header = True, 
                        schema = crimes_schema)



In [52]:
columns = ['ID', 'Description', 'Arrest','PrimaryType', 'Latitude','Longitude','Year','Date']


t1=crimes2.where((crimes2["PrimaryType"] == "HOMICIDE") )\
                                                        .select(columns).toPandas()
t1

,ID,Description,Arrest,PrimaryType,Latitude,Longitude,Year,Date
0,20805,FIRST DEGREE MURDER,False,HOMICIDE,41.728101,-87.656762,2013,02/18/2013 05:15:00 PM
1,995,FIRST DEGREE MURDER,False,HOMICIDE,41.837007,-87.734084,2001,08/05/2001 02:27:00 AM
2,22487,FIRST DEGREE MURDER,False,HOMICIDE,41.804942,-87.662443,2016,05/16/2016 05:05:00 PM
3,2389,FIRST DEGREE MURDER,True,HOMICIDE,41.806423,-87.657615,2003,09/19/2003 08:55:00 PM
4,3799,FIRST DEGREE MURDER,False,HOMICIDE,41.692233,-87.652715,2006,09/21/2006 04:35:00 PM
5,4589,FIRST DEGREE MURDER,False,HOMICIDE,41.749874,-87.552860,2008,06/21/2008 03:36:00 AM
6,2649,FIRST DEGREE MURDER,True,HOMICIDE,41.814020,-87.744522,2004,03/21/2004 06:37:00 PM
7,2313,FIRST DEGREE MURDER,False,HOMICIDE,41.769860,-87.662742,2003,08/03/2003 10:20:00 AM
8,850,FIRST DEGREE MURDER,True,HOMICIDE,41.796620,-87.706090,2001,06/04/2001 11:15:00 AM
9,18995,FIRST DEGREE MURDER,False,HOMICIDE,41.828108,-87.702965,2009,08/05/2009 10:40:00 PM


In [53]:
import folium
# generate a new map
folium_map = folium.Map( location=[41.776940, -87.649569],zoom_start=5)
columns = ['ID', 'Description', 'Arrest','PrimaryType', 'Latitude','Longitude','Year','Date']


t2=crimes2.select(columns).toPandas()
t2.dropna().count()

ID             9909
Description    9909
Arrest         9909
PrimaryType    9909
Latitude       9909
Longitude      9909
Year           9909
Date           9909
dtype: int64

In [54]:
from folium import plugins
from folium.plugins import HeatMap

map_hooray = folium.Map(location=[41.776940, -87.649569],
                    zoom_start = 5) 
# Ensure you're handing it floats
df2=t2[:]
df2['Latitude'] = df2['Latitude'].astype(float)
df2['Longitude'] = df2['Longitude'].astype(float)
df2 = df2[['Latitude', 'Longitude']]# List comprehension to make out list of lists
# Create weight column, using date
df2['Weight'] = t2['Date'].str[0:2]
df2['Weight'] = df2['Weight'].astype(float)
df2 = df2.dropna(axis=0, subset=['Latitude','Longitude', 'Weight'])

# List comprehension to make out list of lists
#this show an intercative plot of data in each month
df2 = [[[row['Latitude'],row['Longitude']] for index, row in df2[df2['Weight'] == i].iterrows()] for i in range(0,13)]

# Plot it on the map
hm = plugins.HeatMapWithTime(df2,
                              auto_play=True,max_opacity=0.8)
hm.add_to(map_hooray)
# Display the map
map_hooray

In [55]:
# generate a new map
folium_map = folium.Map( location=[41.776940, -87.649569],zoom_start=5)

# here we will show our data labled 
t2=t2.dropna()
df2=t2[:500]
df2['Latitude'] = df2['Latitude'].astype(float)
df2['Longitude'] = df2['Longitude'].astype(float)
df2['Date'] = df2['Date'].astype(str)
for index, row in df2.iterrows():
    folium.CircleMarker([row['Latitude'], row['Longitude']],
         popup = ('ID : ' + str(row['ID'] ).capitalize() + '<br>'
                 'Description: ' + str(row['Description']) + '<br>'
                  'Date:' + str(row['Date']) + '<br>'
                 'Arrest: ' + str(row['Arrest']) 
                ),
         color='b',
         key_on = row['ID'],
         fill_color='red',
         fill_opacity=0.7, 
         line_opacity=0.2,
         ).add_to(folium_map)
    
folium_map

C:\Users\rzouga\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\rzouga\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\rzouga\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde